In [46]:
import time
import taxcalc as tc
import pandas as pd
import numpy as np
import helpers
import copy
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, DataRange1d
from bokeh.io import show, output_notebook
from collections import OrderedDict
output_notebook()

Loading BokehJS ...

In [47]:
start_time = time.time()

In [48]:
start_yr = 2018
end_yr = 2027
years = [year for year in range(start_yr, end_yr + 1)]
# read in tax reform and consumption parameters
params = tc.Calculator.read_json_param_objects('tax_reform.json',
                                               assump='welfare_multiples.json')
income_bins = [0, 9999, 19999, 29999, 39999, 49999,
               74999, 99999, 199999, 1000000, 9e99]

# Baseline Calculator

In [49]:
rec_base = tc.Records.cps_constructor()
pol_base = tc.Policy()
consumption_base = tc.Consumption()
consumption_base.update_consumption(params['consumption'])
calc_base = tc.Calculator(records=rec_base, policy=pol_base,
                          consumption=consumption_base)
calc_base.advance_to_year(start_yr)
calc_base.calc_all()
base_tax_rev = calc_base.weighted_total('combined')
base_ben_cost = calc_base.weighted_total('benefit_cost_total')

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


# Tax and Benefit Reform

In [50]:
rec_reform = tc.Records.cps_constructor()
pol_reform = tc.Policy()
# implement tax reform
pol_reform.implement_reform(params['policy'])
calc_reform = tc.Calculator(records=rec_reform, policy=pol_reform)
calc_reform.advance_to_year(start_yr)
calc_reform.calc_all()
base_reform_tax_rev = calc_reform.weighted_total('combined')

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


### Revenue Raised

In [51]:
# modeled benefits and non-modeled benefits
benefits = {'var_names': ['ssi', 'snap', 'mcare', 'mcaid', 'vet', 'e02400', 'e02300', 'housing', 'wic', 'tanf', 'other'],
            'full_names': ['SSI', 'SNAP', 'Medicare', 'Medicaid', 'Veterans Benefits', 'Social Security', 'Unemployment Insurance',
                           'Housing Assistance', 'WIC', 'TANF', 'Non-Modeled'],
            'totals': []}
modeled = 0
for ben in benefits['var_names']:
    try:
        val = calc_base.weighted_total(f'{ben}_ben')
    except AttributeError:
        val = calc_base.weighted_total(ben)
    if ben != 'other':
        modeled += val
    benefits['totals'].append(f'{val * 1e-6:,.2f}')
benefits['var_names'].append('modeled')
benefits['full_names'].append('Modeled')
benefits['totals'].append(f'{modeled * 1e-6:,.2f}')
benefits['var_names'].append('non-modeled')
benefits['full_names'].append('Non-Modeled')
nonmodeled = calc_base.weighted_total('other_ben')
benefits['totals'].append(f'{nonmodeled * 1e-6:,.2f}')
benefits['var_names'].append('total')
benefits['full_names'].append('Total')
total = calc_base.weighted_total('benefit_cost_total')
benefits['totals'].append(f'{total * 1e-6: ,.2f}')
benefitsdf = pd.DataFrame(benefits)
print('Benefit Program Costs')
benefitsdf

Benefit Program Costs


,full_names,totals,var_names
0,SSI,"57,028.62",ssi
1,SNAP,"78,938.18",snap
2,Medicare,"718,291.57",mcare
3,Medicaid,"392,292.19",mcaid
4,Veterans Benefits,"159,827.81",vet
5,Social Security,"929,566.26",e02400
6,Unemployment Insurance,"29,426.28",e02300
7,Housing Assistance,"2,375.34",housing
8,WIC,"3,616.75",wic
9,TANF,"7,762.68",tanf


In [52]:
tax_rev = calc_reform.weighted_total('combined') - calc_base.weighted_total('combined')
ben_rev = calc_base.weighted_total('benefit_cost_total') - calc_reform.weighted_total('benefit_cost_total')
total_rev = tax_rev + ben_rev

In [53]:
print('Revenue Raised for 2018')
print(f'Total Tax Revenue Raised:       ${tax_rev:,.2f}')
print(f'Total Benefit Revenue Raised: ${ben_rev:,.2f}')
print('-' * 51)
print(f'Total Revenue Raised:         ${total_rev:,.2f}')

Revenue Raised for 2018
Total Tax Revenue Raised:       $491,453,931,163.95
Total Benefit Revenue Raised: $2,676,084,867,327.52
---------------------------------------------------
Total Revenue Raised:         $3,167,538,798,491.48


In [54]:
base_reform_mtr = calc_base.mtr_graph(calc_reform)
show(base_reform_mtr)

## Initial UBI

Caculations for a UBI in 2018

In [56]:
u18 = calc_reform.weighted_total('nu18')
abv18 = calc_reform.weighted_total('n1820') + calc_reform.weighted_total('n21')
ubi_18, ubi_u18 = helpers.ubi_amount(total_rev, u18, abv18)
print(f'Base UBI for those under 18: ${ubi_u18:,.2f}')
print(f'Base UBI for those above 18: ${ubi_18:,.2f}')
print('Note: these do not account for additional tax revenue due to the UBI')

Base UBI for those under 18: $5,394.28
Base UBI for those above 18: $10,788.56
Note: these do not account for additional tax revenue due to the UBI


In [57]:
# calculator to find initial UBI tax revenue
rec_init = tc.Records.cps_constructor()
pol_init = tc.Policy()
pol_init.implement_reform(params['policy'])
ubi_reform = {
    2018: {
        '_UBI_u18': [ubi_u18],
        '_UBI_1820': [ubi_18],
        '_UBI_21': [ubi_18]
    }
}
pol_init.implement_reform(ubi_reform)
calc_init = tc.Calculator(records=rec_init, policy=pol_init)
calc_init.advance_to_year(2018)
calc_init.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [58]:
reform_rev = calc_reform.weighted_total('combined')
ubi_tax_rev_init = (calc_init.weighted_total('combined') -
                   reform_rev)
print(f'Initial tax revenue raised by a UBI: ${ubi_tax_rev_init:,.2f}')

Initial tax revenue raised by a UBI: $527,090,568,011.04


## Budget Neutral UBI
Calculations for a UBI that is budget neutral in 2018

In [59]:
# find UBI after accounting for additional tax revenue
ubi_18_final, ubi_u18_final = helpers.ubi_finder(u18, abv18, ubi_18, ubi_u18,
                                                 params, total_rev, reform_rev,
                                                 2018, 100)

2018
Iterations: 15
Revenue Difference: $-47.96


In [60]:
print('Budget Neutral UBI Amounts:')
print(f'\t Those under 18: ${ubi_u18_final:,.2f}')
print(f'\t Those above 18: ${ubi_18_final:,.2f}')

Budget Neutral UBI Amounts:
	 Those under 18: $6,502.06
	 Those above 18: $13,004.11


## Calculations with a UBI for 2018

In [61]:
rec_ubi = tc.Records.cps_constructor()
pol_ubi = tc.Policy()
pol_ubi.implement_reform(params['policy'])
ubi_reform = {
    2018: {
        '_UBI_u18': [ubi_u18_final],
        '_UBI_1820': [ubi_18_final],
        '_UBI_21': [ubi_18_final]
    }
}
pol_ubi.implement_reform(ubi_reform)
calc_ubi = tc.Calculator(records=rec_ubi, policy=pol_ubi)
calc_ubi.advance_to_year(start_yr)
calc_ubi.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [62]:
total_ubi_cost = calc_ubi.weighted_total('ubi')
tax_rev_ubi = (calc_ubi.weighted_total('combined') - base_tax_rev)
ben_rev_ubi = (base_ben_cost - calc_ubi.weighted_total('benefit_cost_total'))
total_ubi_rev = tax_rev_ubi + ben_rev_ubi
diff = total_ubi_cost - total_ubi_rev
print(f'Total Cost of the UBI: ${total_ubi_cost:,.2f}')
print(f'Cost Difference: ${diff:,.2f}')

Total Cost of the UBI: $3,818,028,958,479.40
Cost Difference: $-47.96


In [63]:
# UBI table data prep
table_data = calc_ubi.dataframe(['ubi', 'e00200', 'nu18', 'n1820', 'n21',
                                 'combined', 's006', 'benefit_cost_total',
                                 'benefit_value_total', 'elderly_dependent',
                                 'age_head', 'age_spouse'])
mtrs = calc_ubi.mtr()
table_data['pmtr'] = mtrs[0]
table_data['imtr'] = mtrs[1]
table_data['cmtr'] = mtrs[2]
table_data['ppl'] = table_data['nu18'] + table_data['n1820'] + table_data['n21']
table_data['ubi/person'] = table_data['ubi'] / table_data['ppl']
tax_change = calc_ubi.array('combined') - calc_base.array('combined')
table_data['tax_change'] = tax_change
ben_change = (calc_ubi.array('benefit_cost_total') -
              calc_base.array('benefit_cost_total'))
table_data['ben_change'] = ben_change
ben_change_welfare = (table_data['benefit_value_total'] -
                      calc_base.array('benefit_value_total'))
table_data['ben_change_welfare'] = ben_change_welfare
combined_change = (table_data['ubi'] -
                   table_data['tax_change'] +
                   table_data['ben_change'])
table_data['combined_change'] = combined_change
combined_change_welfare = (table_data['ubi'] -
                           table_data['tax_change'] +
                           table_data['ben_change_welfare'])
table_data['combined_change_welfare'] = combined_change_welfare
table_data['65_plus'] = np.where((table_data['elderly_dependent'] == 1) |
                                  (table_data['age_head'] >= 65) |
                                  (table_data['age_spouse'] >= 65), 1, 0)

## 2018 UBI Tables

### All Tax Units

In [64]:
table_all = helpers.table(table_data, 'e00200', income_bins)
table_all.to_csv('table_all.csv', index=False)
table_all

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,61.87,1.568728,"12,318",14.2%,13.3%,27.4%,"3,082","18,663","-31,535","-25,956","-15,954","-10,375"
1,10,14.41,1.777568,"12,058",14.2%,12.2%,26.4%,"5,517","20,060","-11,297","-8,692","3,246","5,851"
2,20,15.25,1.872203,"12,027",14.2%,14.8%,29.0%,"6,169","21,042","-8,891","-6,787","5,982","8,086"
3,30,12.41,1.918864,"12,066",14.2%,18.2%,32.4%,"6,460","21,678","-7,314","-5,575","7,904","9,643"
4,40,10.62,2.026123,"12,044",14.2%,18.9%,33.1%,"7,006","22,849","-6,879","-5,403","8,964","10,439"
5,50,19.00,2.174613,"12,005",14.2%,20.4%,34.6%,"8,104","24,545","-5,920","-4,787","10,521","11,655"
6,75,12.31,2.439413,"11,880",14.2%,21.2%,35.4%,"10,407","27,332","-4,606","-3,786","12,320","13,140"
7,100,18.66,2.734243,"11,703",12.6%,22.3%,34.9%,"13,173","30,369","-3,959","-3,364","13,237","13,831"
8,200,5.24,2.927686,"11,541",7.9%,27.4%,35.2%,"17,469","32,135","-3,667","-3,053","10,999","11,613"
9,"1,000",0.11,2.874847,"11,514",6.3%,36.0%,42.3%,"31,649","31,272","-4,236","-3,459","-4,613","-3,836"


### Tax Units with Someone 65 or Older

In [65]:
elderly_data = copy.deepcopy(table_data[table_data['65_plus'] == 1])

In [66]:
elderly_table = helpers.table(elderly_data, 'e00200', income_bins)
elderly_table.to_csv('elderly_table.csv', index=False)
elderly_table

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,27.44,1.499430,"12,879",14.2%,15.6%,29.8%,"2,805","19,165","-46,471","-40,056","-30,111","-23,696"
1,10,1.41,1.742564,"12,820",14.2%,14.4%,28.6%,"4,622","22,073","-40,124","-34,879","-22,673","-17,427"
2,20,1.21,1.827770,"12,767",14.2%,15.5%,29.7%,"5,104","22,952","-37,154","-32,371","-19,307","-14,523"
3,30,0.93,1.902106,"12,748",14.2%,17.3%,31.5%,"5,370","23,737","-33,274","-29,136","-14,907","-10,769"
4,40,0.82,1.938251,"12,748",14.2%,18.2%,32.4%,"5,456","24,265","-35,824","-31,408","-17,015","-12,598"
5,50,1.55,1.970146,"12,764",14.2%,20.6%,34.8%,"6,150","24,725","-33,193","-29,234","-14,618","-10,658"
6,75,0.87,2.082399,"12,720",14.2%,21.3%,35.4%,"7,792","25,979","-28,792","-25,824","-10,605","-7,637"
7,100,1.33,2.170850,"12,714",12.3%,22.9%,35.2%,"9,068","27,047","-28,300","-25,414","-10,321","-7,435"
8,200,0.41,2.251385,"12,658",7.9%,28.4%,36.3%,"12,174","27,843","-24,483","-22,119","-8,814","-6,450"
9,"1,001",0.01,2.099700,"12,611",6.9%,35.9%,42.7%,"21,990","26,045","-22,586","-21,231","-18,531","-17,176"


### Tax Units without Someone 65 or Older

In [67]:
non_elderly_data = copy.deepcopy(table_data[table_data['65_plus'] == 0])

In [68]:
non_elderly_table = helpers.table(non_elderly_data, 'e00200', income_bins)
non_elderly_table.to_csv('non_elderly_table.csv', index=False)
non_elderly_table

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,34.43,1.623964,"11,872",14.2%,11.4%,25.6%,"3,303","18,263","-19,629","-14,718","-4,669",243
1,10,13.00,1.781374,"11,975",14.2%,12.0%,26.2%,"5,614","19,841","-8,164","-5,845","6,063","8,382"
2,20,14.05,1.876024,"11,963",14.2%,14.8%,29.0%,"6,260","20,877","-6,460","-4,586","8,157","10,031"
3,30,11.48,1.920224,"12,010",14.2%,18.3%,32.5%,"6,548","21,511","-5,207","-3,662","9,756","11,300"
4,40,9.81,2.033449,"11,985",14.2%,18.9%,33.1%,"7,136","22,731","-4,466","-3,236","11,130","12,360"
5,50,17.44,2.192809,"11,937",14.2%,20.4%,34.6%,"8,278","24,529","-3,493","-2,611","12,758","13,640"
6,75,11.44,2.466706,"11,816",14.2%,21.2%,35.4%,"10,607","27,436","-2,757","-2,101","14,072","14,728"
7,100,17.33,2.777592,"11,625",12.6%,22.3%,34.9%,"13,489","30,624","-2,086","-1,668","15,049","15,468"
8,200,4.83,2.985243,"11,446",7.8%,27.3%,35.1%,"17,920","32,500","-1,896","-1,431","12,685","13,150"
9,"1,000",0.10,2.950728,"11,407",6.3%,36.0%,42.3%,"32,595","31,784","-2,440","-1,719","-3,250","-2,530"


In [69]:
reform_ubi = calc_reform.mtr_graph(calc_ubi)
show(reform_ubi)

## 10-Year Calculations with Initial UBI

In [71]:
rec_10yr = tc.Records.cps_constructor()
pol_10yr = tc.Policy()
pol_10yr.implement_reform(params['policy'])
# UBI reform defined above
pol_10yr.implement_reform(ubi_reform)
calc_10yr = tc.Calculator(records=rec_10yr, policy=pol_10yr)
calc_10yr.advance_to_year(start_yr)
calc_10yr.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [72]:
# dictionaries for storing data
var_list = ['ubi', 'combined', 'benefit_cost_total', 'benefit_value_total']
ubi_dict = {'year': [2018]}
base_dict = {'year': [2018]}
for var in var_list:
    ubi_dict[var] = [calc_10yr.weighted_total(var)]
    base_dict[var] = [calc_base.weighted_total(var)]

In [73]:
# 10 year calculations
for year in range(start_yr + 1, end_yr + 1):
    calc_base.advance_to_year(year)
    calc_base.calc_all()
    calc_10yr.advance_to_year(year)
    calc_10yr.calc_all()
    for var in var_list:
        ubi_dict[var].append(calc_10yr.weighted_total(var))
        base_dict[var].append(calc_base.weighted_total(var))
    ubi_dict['year'].append(year)
    base_dict['year'].append(year)

In [74]:
# convert to data frames for processing
ubi_df = pd.DataFrame(ubi_dict)
base_df = pd.DataFrame(base_dict)
ubi_df['ubi_cumsum'] = ubi_df['ubi'].cumsum()

## Revenue Effects

In [75]:
total_ubi = ubi_df['ubi'].sum()
tax_revenue_change = ubi_df['combined'].sum() - base_df['combined'].sum()
benefit_savings = base_df['benefit_cost_total'].sum() - ubi_df['benefit_cost_total'].sum()
total_revenue_available = tax_revenue_change + benefit_savings
ten_yr_diff = total_ubi - total_revenue_available

print(f'Tax Revenue Change:{" " * 5} ${tax_revenue_change:,.2f}')
print(f'Benefit Cost Savings:{" " * 3} ${benefit_savings:,.2f}')
print(f'Total Available Revenue: ${total_revenue_available:,.2f}')
print(f'Total UBI Cost:{" " * 9} ${total_ubi:,.2f}')
print('-' * 47)
print(f'Difference:{" " * 13} ${ten_yr_diff:,.2f}')

Tax Revenue Change:      $14,292,683,052,864.35
Benefit Cost Savings:    $33,140,130,705,886.39
Total Available Revenue: $47,432,813,758,750.75
Total UBI Cost:          $44,743,914,953,356.54
-----------------------------------------------
Difference:              $-2,688,898,805,394.21


In [76]:
# data for plotting
tax_rev_series = ubi_df['combined'] - base_df['combined']
benefit_savings_series = base_df['benefit_cost_total'] - ubi_df['benefit_cost_total']
available_revenue = tax_rev_series + benefit_savings_series
ten_yr_series = ubi_df['ubi'] - tax_rev_series - benefit_savings_series
available_rev_cumsum = available_revenue.cumsum()
cummulative_diff = ubi_df['ubi_cumsum'] - available_rev_cumsum

cds = ColumnDataSource({'total_revenue_avail': available_revenue,
                        'ubi_cost': ubi_df['ubi'],
                        'tax_revenue_change': tax_rev_series,
                        'benefit_savings': benefit_savings_series,
                        'ten_yr_diff': ten_yr_series,
                        'year': ubi_df['year'],
                        'ubi_cumsum': ubi_df['ubi_cumsum'],
                        'total_revenue_avail_cumsum': available_rev_cumsum,
                        'cummulative_diff': cummulative_diff})

In [77]:
f = figure(title='UBI Cost and Available Revenue',
           width=650, height=500, x_range=DataRange1d())
f.line(x='year', y='total_revenue_avail', line_width=2, legend='Available Revenue', source=cds)
f.line(x='year', y='ubi_cost', color='red', line_width=2, legend='UBI Cost', source=cds)
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.legend.location = 'top_left'
show(f)

# Ten-Year Budget Neutrality

In [79]:
ubi_18_tenyr, ubi_u18_tenyr = helpers.ten_year(params, 13004.11, start_yr, end_yr, 2000000)

-2688899438986.914
Entered Loop
Iterations: 25


In [80]:
print('10-Year Budget Neutral UBI Amounts:')
print(f'\t Those under 18: ${ubi_u18_tenyr:,.2f}')
print(f'\t Those above 18: ${ubi_18_tenyr:,.2f}')

10-Year Budget Neutral UBI Amounts:
	 Those under 18: $6,991.82
	 Those above 18: $13,983.64


In [81]:
end_time = time.time()

In [82]:
(end_time - start_time) / 60

25.81267048517863

In [83]:
# Calculator with 10-year budget neutral UBI
rec_final = tc.Records.cps_constructor()
pol_final = tc.Policy()
pol_final.implement_reform(params['policy'])
ubi_final = {
    start_yr: {
        '_UBI_u18': [ubi_u18_tenyr],
        '_UBI_1820': [ubi_18_tenyr],
        '_UBI_21': [ubi_18_tenyr]
    }
}
pol_final.implement_reform(ubi_final)
calc_final = tc.Calculator(records=rec_final, policy=pol_final)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Revenue Effects

In [84]:
# collect variables for the final calculator
final_dict = {'year': [], 'ubi': [], 'combined': [],
              'benefit_cost_total': [], 'benefit_value_total': []}
for year in range(start_yr, end_yr + 1):
    final_dict['year'].append(year)
    calc_final.advance_to_year(year)
    calc_final.calc_all()
    for var in var_list:
        final_dict[var].append(calc_final.weighted_total(var))

In [85]:
final_df = pd.DataFrame(final_dict)

In [86]:
total_ubi_final = final_df['ubi'].sum()
tax_revenue_change_final = final_df['combined'].sum() - base_df['combined'].sum()
benefit_savings_final = base_df['benefit_cost_total'].sum() - final_df['benefit_cost_total'].sum()
total_revenue_available_final = tax_revenue_change_final + benefit_savings_final
ten_yr_diff_final = total_ubi_final - total_revenue_available_final

print(f'Tax Revenue Change:{" " * 5} ${tax_revenue_change_final:,.2f}')
print(f'Benefit Cost Savings:{" " * 3} ${benefit_savings_final:,.2f}')
print(f'Total Available Revenue: ${total_revenue_available_final:,.2f}')
print(f'Total UBI Cost:{" " * 9} ${total_ubi_final:,.2f}')
print('-' * 47)
print(f'Difference:{" " * 21} ${ten_yr_diff_final:,.2f}')

Tax Revenue Change:      $14,974,094,699,227.16
Benefit Cost Savings:    $33,140,130,705,886.39
Total Available Revenue: $48,114,225,405,113.55
Total UBI Cost:          $48,114,223,439,314.21
-----------------------------------------------
Difference:                      $-1,965,799.34


In [87]:
# format data for plotting
tax_rev_series_final = final_df['combined'] - base_df['combined']
benefit_savings_series_final = base_df['benefit_cost_total'] - final_df['benefit_cost_total']
available_revenue_final = tax_rev_series_final + benefit_savings_series_final
ten_yr_series_final = final_df['ubi'] - tax_rev_series_final - benefit_savings_series_final
available_rev_cumsum_final = available_revenue_final.cumsum()

cds_final = ColumnDataSource({'total_revenue_avail': available_revenue_final,
                              'ubi_cost': final_df['ubi'],
                              'tax_revenue_change': tax_rev_series_final,
                              'benefit_savings': benefit_savings_series_final,
                              'ten_yr_diff': ten_yr_series_final,
                              'year': final_df['year'],
                              'total_revenue_avail_cumsum': available_rev_cumsum_final})

In [88]:
f = figure(title='UBI Cost and Available Revenue',
           width=650, height=500, x_range=DataRange1d())
f.line(x='year', y='total_revenue_avail', line_width=2, legend='Available Revenue', source=cds_final)
f.line(x='year', y='ubi_cost', color='red', line_width=2, legend='UBI Cost', source=cds_final)
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.legend.location = 'top_left'
show(f)

In [90]:
tc.__version__

'0.16.2'